# 가중치 가지치기 및 양자화를 통한 모델 최적화

## 요약

1. MNIST 데이터를 tf.keras 라이브러리를 이용해 모델링한다.
2. Pruning API를 적용하여 모델을 미세조정하고 정확도를 확인한다.
3. 가지치기에서 3배 더 작은 TF와 TFLite 모델을 만든다.
4. 가지치기와 훈련 후 양자화를 거쳐 10배 더 작은 TFLite 모델을 만든다.
5. 최적화된 TF 및 TFLite 모델의 정확도를 확인한다.

## 준비

### 라이브러리 불러오기

In [47]:
import os
import zipfile
import tempfile

import numpy as np
import tensorflow as tf

import tensorflow_model_optimization as tfmot

### TensorFlow 로그 제어

In [2]:
from freeman.utils.support_tf import LogLevelManager as llm
llm.set(2)

## 일반적인 모델 훈련

### 데이터 불러오기

In [3]:
mnist = tf.keras.datasets.mnist
(train_x, train_y), (test_x, test_y) = mnist.load_data()

In [5]:
train_x.shape, test_x.shape

((60000, 28, 28), (10000, 28, 28))

### 정규화

In [6]:
train_x, test_x = train_x / 255., test_x / 255.

### 모델 생성

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28, 28)),
    tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(filters=12, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

In [8]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 12)        120       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 12)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 2028)              0         
                                                                 
 dense (Dense)               (None, 10)                20290     
                                                                 
Total params: 20,410
Trainable params: 20,410
Non-trainable params: 0
____________________________________________________

### 모델 훈련

In [9]:
%%time
model.fit(train_x, train_y, epochs=5, validation_split=.1, verbose=0)

CPU times: user 1min 18s, sys: 11.4 s, total: 1min 30s
Wall time: 53.7 s


### 모델 평가

In [10]:
_, accuracy = model.evaluate(test_x, test_y, verbose=0)
accuracy

0.9783999919891357

### 모델 저장

In [26]:
base_model_path = os.path.join(os.path.expanduser("~"), "temp")

normal_model = os.path.join(base_model_path, "normal_model.h5")
tf.keras.models.save_model(model, normal_model, include_optimizer=False)

normal_model_size = os.path.getsize(normal_model)
f"{normal_model_size} bytes"

'98968 bytes'

## 가지치기(Pruning)을 통한 모델 미세조정

* 가지치기를 전체 모델에 적용하고 모델 요약에서 이를 확인한다.
* 이 예제는 50% 희소성(가중치가 0인 50%)으로 모델을 시작하고, 80% 희소성으로 종료한다.
* <b>모델 정확도를 높이기 위해 일부 레이어를 잘라낼 수 도 있다.</b>

### 가지치기 옵션 정의

In [28]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# 여기서 정의된 batch_size, epochs 등은 위에서 학습된 모델에서 사용된 것이 아니라,
# 가지치기를 위한 파라미터임.
batch_size = 128
epochs = 2
validation_split = 0.1

num_train_x = train_x.shape[0] * (1 - validation_split)
end_step = np.ceil(num_train_x / batch_size).astype(np.int32) * epochs  # np.ceil(반올림)
num_train_x, end_step

(54000.0, 844)

In [31]:
pruning_params = {
    "pruning_schedule":
        tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                             final_sparsity=0.80,
                                             begin_step=0,
                                             end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

model_for_pruning.compile(optimizer="adam",
                          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                          metrics=["accuracy"])

### 가지치기된 모델 요약

In [32]:
model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshape  (None, 28, 28, 1)        1         
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_conv2d   (None, 26, 26, 12)       230       
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_max_poo  (None, 13, 13, 12)       1         
 ling2d (PruneLowMagnitude)                                      
                                                                 
 prune_low_magnitude_flatten  (None, 2028)             1         
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_dense (  (None, 10)               4

### 가지치기 모델 훈련

In [35]:
%%time

logdir = tempfile.mkdtemp()
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),               # 훈련중 사용
    tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),  # 진행사항 추적 및 디버깅용 로그 제공
]

model_for_pruning.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, 
                      validation_split=validation_split, callbacks=callbacks)

Epoch 1/2
422/422 [==============================] - 6s 11ms/step - loss: 0.1197 - accuracy: 0.9670 - val_loss: 0.1641 - val_accuracy: 0.9643
Epoch 2/2
422/422 [==============================] - 4s 10ms/step - loss: 0.1377 - accuracy: 0.9658 - val_loss: 0.0981 - val_accuracy: 0.9768
CPU times: user 20.2 s, sys: 3.37 s, total: 23.6 s
Wall time: 10.9 s


In [42]:
!tensorboard --logdir={logdir}


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.10.1 at http://localhost:6007/ (Press CTRL+C to quit)
^C


### 가지치기 모델 평가

In [36]:
_, accuracy_for_pruning = model_for_pruning.evaluate(test_x, test_y, verbose=0)
accuracy_for_pruning

0.9678000211715698

### 가지치기 모델 축소

#### 압축 가능한 일반 모델로 변경

In [37]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

#### 압축 가능한 일반 모델 저장

* 변경 전/후의 모델 크기는 동일하다.

In [40]:
export_model = os.path.join(base_model_path, "export_model.h5")
tf.keras.models.save_model(model_for_export, export_model, include_optimizer=False)

export_model_size = os.path.getsize(export_model)
f"{export_model_size} bytes"

'98968 bytes'

#### 압축 가능한 TFLite 모델로 변경

In [43]:
model_for_lite = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
model_lite = model_for_lite.convert()

INFO:tensorflow:Assets written to: /tmp/tmp4c577ohs/assets


In [45]:
file_lite = os.path.join(base_model_path, "lite_model.tflite")
with open(file_lite, "wb") as f:
    f.write(model_lite)

In [46]:
lite_model_size = os.path.getsize(file_lite)
f"{lite_model_size} bytes"

'84500 bytes'

## 모델 압축

In [48]:
def get_gzipped_model_size(file):
    _, zipped_file = tempfile.mkstemp(".zip")
    with zipfile.ZipFile(zipped_file, "w", compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)
    return os.path.getsize(zipped_file)